In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
#loading modules
import math, random, os, re, time
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import layers, Model
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
import gc
import cv2
import seaborn as sns
from kaggle_datasets import KaggleDatasets
from tqdm import tqdm


In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.


AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
#REPLICAS = 8
print(f'REPLICAS: {REPLICAS}')


In [ ]:
#loading data
dirname='../input/siim-isic-melanoma-classification/'
train = pd.read_csv(dirname+'train.csv')
test = pd.read_csv(dirname + 'test.csv')
print(train.head())
print(len(train))
print(len(test))
print(train['target'].value_counts())


In [ ]:
sns.countplot(train['target'])


In [ ]:
GCS_PATH = KaggleDatasets().get_gcs_path()
train_filenames = tf.io.gfile.glob(GCS_PATH + '/tfrecords/train*.tfrec')
test_filenames = tf.io.gfile.glob(GCS_PATH + '/tfrecords/test*.tfrec')


In [ ]:
!gsutil ls $GCS_PATH


In [ ]:
train_filenames , valid_filenames = train_test_split(train_filenames , test_size=0.2,shuffle=True)


In [ ]:
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
IMAGE_SIZE = [1024,1024]
AUTO = tf.data.experimental.AUTOTUNE
imSize = 1024


In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    image = tf.image.resize(image, [imSize,imSize])
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "target": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['target'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "image_name": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['image_name']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_saturation(image, 0, 2)
    # image = tf.image.random_hue(image,0.15)
    return image, label   

def get_training_dataset():
    dataset = load_dataset(train_filenames, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_val_dataset():
    dataset = load_dataset(valid_filenames, labeled=True)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(train_filenames)
NUM_TEST_IMAGES = count_data_items(valid_filenames)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
print('Dataset: {} training images, {} labeled validation images'.format(NUM_TRAINING_IMAGES, NUM_TEST_IMAGES))


In [ ]:
for image, label in get_training_dataset().take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy())
# print("Test data shapes:")


In [ ]:
def res_block(X_in, channels):
    X = layers.Conv2D(channels, (3,3), strides=(1,1), padding='same' )(X_in)
    X = layers.BatchNormalization()(X)
    X = layers.LeakyReLU()(X)
    
    X = layers.Conv2D(channels, (3,3), strides=(1,1), padding='same')(X)
    X = layers.BatchNormalization()(X)
    X = layers.Add()([X, X_in])
    X = layers.LeakyReLU()(X)
    
    return X


In [ ]:
#model
def my_model():
    X_in = layers.Input((1024, 1024, 3))
    
    X = layers.AveragePooling2D(pool_size=(2, 2), strides=2, name='avg_pool1')(X_in)
    
    X = layers.Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv1')(X)
    X = layers.BatchNormalization()(X)
    X = layers.Activation('relu')(X)

    X = res_block(X, 64)
    
    X = layers.MaxPool2D(pool_size=(2, 2), strides=2, name='max_pool1')(X)
    X = layers.MaxPool2D(pool_size=(2, 2), strides=2, name='max_pool1.2')(X)

    X = layers.Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv2')(X)
    X = layers.BatchNormalization()(X)
    X = layers.Activation('relu')(X)
    
    X = res_block(X, 128)
    
    X = layers.MaxPool2D(pool_size = (2,2), strides=2, name='max_pool2')(X)

    X = layers.Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv3')(X)
    X = layers.BatchNormalization()(X)
    X = layers.Activation('relu')(X)

    X = res_block(X, 256)
    
    X = layers.MaxPool2D(pool_size = (2,2), strides=2, name='max_pool3')(X)

    X = res_block(X, 256)
    
    X = layers.MaxPool2D(pool_size = (2,2), strides=2, name='max_pool4')(X)
    
    X = res_block(X, 256)
    
    X = layers.MaxPool2D(pool_size = (2,2), strides=2, name='max_pool5')(X)
    
    X = layers.Flatten()(X)
    X = layers.Dense(4096, activation='relu', name='fc1')(X)
    X = layers.Dense(1024, activation='relu', name='fc2')(X)
    X_out = layers.Dense(1, activation='sigmoid', name='answer')(X)

    model = Model(inputs=X_in, outputs=X_out, name='pinnet')
    
    return model


In [ ]:
with strategy.scope():
    model = my_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
def callback():
    cb = []
    reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.3, patience=5,
                                   verbose=1, mode='auto',
                                   epsilon=0.0001, cooldown=1, min_lr=0.00001)
    cb.append(reduceLROnPlat)
    return cb


In [ ]:
cb = callback()
#train and validate
epochs = 10
history = model.fit(get_training_dataset(), epochs=epochs, verbose=True, steps_per_epoch=NUM_TRAINING_IMAGES // BATCH_SIZE,
                    validation_data = get_val_dataset(), validation_steps =NUM_TEST_IMAGES//BATCH_SIZE, callbacks=cb) # callbacks = cb,


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
num_test_images = count_data_items(test_filenames)
num_test_images


In [ ]:
def get_test_dataset(ordered=False):
    dataset = load_dataset(test_filenames, labeled=False,ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

test_dataset = get_test_dataset(ordered=True)


In [ ]:
print('Computing predictions...')
test_images_ds = test_dataset.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds).flatten()
print(probabilities)


print('Generating submission.csv file...')
test_ids_ds = test_dataset.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(num_test_images))).numpy().astype('U') # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, probabilities]), fmt=['%s', '%f'], delimiter=',', header='image_name,target', comments='')
